In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob # Import the glob module
# Define dataset path
dataset_path = "/content/drive/MyDrive/Filtered Data"  # Update this path

# Get all text files inside 12 subfolders
txt_files = glob.glob(os.path.join(dataset_path, "**/*.txt"), recursive=True)

print(f"Found {len(txt_files)} dataset files.")

Found 56 dataset files.


In [ ]:
import pandas as pd


In [ ]:
import os
import pandas as pd

# Path to your dataset folder in Google Drive
base_path = "/content/drive/MyDrive/Filtered Data"

# List to store file paths
txt_files = []

# Traverse the dataset folders
for folder in sorted(os.listdir(base_path)):
    folder_path = os.path.join(base_path, folder)

    if os.path.isdir(folder_path):  # Ensure it's a folder
        if folder == "008":  # Special handling for folder 8
            for subfolder in ["OFF_1", "OFF_2"]:
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.exists(subfolder_path):  # Check if subfolder exists
                    for file in os.listdir(subfolder_path):
                        if file.endswith(".txt"):
                            txt_files.append(os.path.join(subfolder_path, file))
        else:  # All other folders where .txt files are directly present
            for file in os.listdir(folder_path):
                if file.endswith(".txt"):
                    txt_files.append(os.path.join(folder_path, file))

# Verify collected files
print(f"Total .txt files found: {len(txt_files)}")


Total .txt files found: 56


In [ ]:
# Columns related to acceleration & gyroscope (32-59)
gait_columns = list(range(32, 59))  # Includes x, y, z accelerometer + gyro
label_column = [60]  # Freezing of Gait (FoG) Label

# Initialize list for storing processed data
all_data = []

# Read all files
for file in txt_files:
    df = pd.read_csv(file, header=None)  # No header in dataset
    gait_data = df[gait_columns + label_column]  # Extract only gait & labels
    all_data.append(gait_data)

# Merge all files
gait_df = pd.concat(all_data, ignore_index=True)

print("Gait Data Shape:", gait_df.shape)


Gait Data Shape: (6211056, 28)


In [ ]:
import torch

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert gait data to PyTorch tensor and move to GPU
gait_tensor = torch.tensor(gait_df.values, dtype=torch.float32, device=device)

window_size = 1000  # 2 sec (500Hz)
step_size = 500     # 50% overlap

# Batch-wise segmentation for speed
num_samples = (gait_tensor.shape[0] - window_size) // step_size

# Preallocate tensors
X = torch.zeros((num_samples, window_size, gait_tensor.shape[1] - 1), device=device)
y = torch.zeros((num_samples,), dtype=torch.long, device=device)

# Fill tensors
for i in range(num_samples):
    start = i * step_size
    X[i] = gait_tensor[start : start + window_size, :-1]  # All gait features
    y[i] = gait_tensor[start + window_size - 1, -1]  # Last label in window

print("Segmented Data Shape:", X.shape, "Labels Shape:", y.shape)


Segmented Data Shape: torch.Size([12420, 1000, 27]) Labels Shape: torch.Size([12420])


In [ ]:
import torch
import numpy as np

# Move data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert DataFrame to NumPy
gait_data = gait_df.iloc[:, :-1].values  # Features
labels = gait_df.iloc[:, -1].values  # Labels

# Convert to PyTorch tensors and move to GPU
gait_tensor = torch.tensor(gait_data, dtype=torch.float32, device=device)
labels_tensor = torch.tensor(labels, dtype=torch.int64, device=device)

# Segmentation parameters
window_size = 1000  # 2 seconds
step_size = 500

segments = []
labels = []

# GPU batch segmentation
for i in range(0, len(gait_tensor) - window_size, step_size):
    segment = gait_tensor[i:i+window_size]  # Feature segment
    label = labels_tensor[i+window_size-1]  # Label at window end
    segments.append(segment)
    labels.append(label)

# Stack as tensors
X = torch.stack(segments).to(device)
y = torch.tensor(labels, dtype=torch.int64, device=device)

print("Segmented Data Shape:", X.shape)


Segmented Data Shape: torch.Size([12421, 1000, 27])


In [24]:
import torch
import torch.nn as nn

class CNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CNN_LSTM, self).__init__()

        # CNN Layers
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # LSTM Layers
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Convert to (batch, channels, time) for CNN
        x = self.pool(self.relu(self.conv1(x)))  # CNN Feature Extraction
        x = x.permute(0, 2, 1)  # Back to (batch, time, features)

        lstm_out, _ = self.lstm(x)
        lstm_features = lstm_out[:, -1, :]  # Extract last LSTM hidden state

        output = self.fc(lstm_features)  # Final classification output
        return output

    # New method to extract feature vectors
    def extract_features(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool(self.relu(self.conv1(x)))
        x = x.permute(0, 2, 1)

        lstm_out, _ = self.lstm(x)
        lstm_features = lstm_out[:, -1, :]  # Extract feature vector before final FC layer
        return lstm_features


In [25]:
import torch.optim as optim

# Define model parameters
input_size = 27  # Number of gait features
hidden_size = 128
num_layers = 2
output_size = 1  # Binary classification (FoG or no FoG)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(input_size, hidden_size, num_layers, output_size).to(device)

# Define Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

print("CNN-LSTM Training Completed ✅")


Epoch 1, Loss: 0.575844870790408
Epoch 2, Loss: 0.5622298310232315
Epoch 3, Loss: 0.5670619021466308
Epoch 4, Loss: 0.5626194989757906
Epoch 5, Loss: 0.5573569317722628
CNN-LSTM Training Completed ✅


In [27]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X).squeeze()
        preds = torch.round(torch.sigmoid(outputs))  # Convert logits to binary predictions
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}% ✅")


Test Accuracy: 77.79% ✅


In [28]:
import numpy as np

# Ensure model is in evaluation mode
model.eval()

feature_vectors = []

with torch.no_grad():
    for batch_X, _ in test_loader:  # No need for labels here
        batch_X = batch_X.to(device)
        features = model.extract_features(batch_X)  # Extract features from LSTM layer
        feature_vectors.append(features.cpu().numpy())

# Convert to numpy array
feature_vectors = np.vstack(feature_vectors)

# Save feature vectors to a file
np.save('gait_feature_vectors.npy', feature_vectors)
print("Feature vectors saved successfully! ✅")


Feature vectors saved successfully! ✅


In [29]:
from google.colab import files
files.download('gait_feature_vectors.npy')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
print(feature_vectors.shape)  # Should be (num_samples, feature_dim)
print("Example feature vector:", feature_vectors[0][:10])  # Print first 10 values of first vector


(2485, 128)
Example feature vector: [-0.03943942 -0.2911995   0.2068045  -0.04807587  0.02643567  0.2355742
  0.0088343   0.03526265 -0.00297631  0.07361782]
